In [1]:
import numpy as np
import os
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from PIL import Image
import pandas as pd


In [2]:
from PIL import Image
import numpy as np
import os
import matplotlib.pyplot as plt
import pickle

# Parameters
train_folder = "../Datasets/aqueduct, industrial_area, patio data sets/train/"
test_folder = "../Datasets/aqueduct, industrial_area, patio data sets/train/"
class_folders = ["aqueduct", "industrial_area", "patio"]
patch_size = 32
bins = 8
k = 32  # Number of clusters for K-means

# Patch extraction function
def extract_patches(image, patch_size=32):
    patches = []
    width, height = image.size
    for i in range(0, height, patch_size):
        for j in range(0, width, patch_size):
            patch = image.crop((j, i, j + patch_size, i + patch_size))
            if patch.size == (patch_size, patch_size):
                patches.append(np.array(patch))
    return patches

# Color histogram extraction function
def color_histogram(patch, bins=8):
    hist_r = np.histogram(patch[:, :, 0], bins=bins, range=(0, 256))[0]
    hist_g = np.histogram(patch[:, :, 1], bins=bins, range=(0, 256))[0]
    hist_b = np.histogram(patch[:, :, 2], bins=bins, range=(0, 256))[0]
    return np.concatenate([hist_r, hist_g, hist_b])

# K-means implementation from scratch
def initialize_centroids(features, k):
    indices = np.random.choice(len(features), k, replace=False)
    return features[indices]

def assign_clusters(features, centroids):
    distances = np.linalg.norm(features[:, np.newaxis] - centroids, axis=2)
    return np.argmin(distances, axis=1)

def update_centroids(features, labels, k):
    new_centroids = np.zeros((k, features.shape[1]))
    for i in range(k):
        points_in_cluster = features[labels == i]
        if len(points_in_cluster) > 0:
            new_centroids[i] = np.mean(points_in_cluster, axis=0)
    return new_centroids

def kmeans(features, k, max_iter=100, tol=1e-4):
    centroids = initialize_centroids(features, k)
    for _ in range(max_iter):
        labels = assign_clusters(features, centroids)
        new_centroids = update_centroids(features, labels, k)
        if np.linalg.norm(new_centroids - centroids) < tol:
            break
        centroids = new_centroids
    return centroids, labels

# BoVW representation for an image
def bovw_representation(image_patches, centroids):
    features = [color_histogram(patch) for patch in image_patches]
    features = np.array(features)
    labels = assign_clusters(features, centroids)
    bovw_hist = np.bincount(labels, minlength=len(centroids))
    return bovw_hist / len(features)

# Save data function
def save_data(data, filename):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)

# Load data function
def load_data(filename):
    if os.path.exists(filename):
        with open(filename, 'rb') as f:
            return pickle.load(f)
    return None

# Process images and save BoVW representations for each class separately
def process_images(folder, class_folders, centroids, save_folder, prefix):
    for i, class_name in enumerate(class_folders, 1):
        class_folder = os.path.join(folder, class_name)
        class_data = []
        
        for img_name in os.listdir(class_folder):
            img_path = os.path.join(class_folder, img_name)
            img = Image.open(img_path)
            patches = extract_patches(img, patch_size)
            bovw_vector = bovw_representation(patches, centroids)
            class_data.append((img_name, bovw_vector, class_name))
        
        # Save each class data in a separate file for each image
        for j, data in enumerate(class_data, 1):
            img_name, bovw_vector, class_name = data
            save_data(bovw_vector, os.path.join(save_folder, f'{prefix}_data{i}_{j}.pkl'))

# Main workflow
def main():
    # Check if BoVW data exists; if not, process it
    if not os.path.exists('train_data1_1.pkl'):
        # Collect all patches from training images to fit K-means
        all_train_features = []
        for class_name in class_folders:
            class_folder = os.path.join(train_folder, class_name)
            for img_name in os.listdir(class_folder):
                img_path = os.path.join(class_folder, img_name)
                img = Image.open(img_path)
                patches = extract_patches(img, patch_size)
                for patch in patches:
                    all_train_features.append(color_histogram(patch))

        all_train_features = np.array(all_train_features)
        print("Total training patches:", len(all_train_features))

        # Apply K-means clustering
        centroids, _ = kmeans(all_train_features, k)
        print("K-means clustering completed.")

        # Process train and test datasets and save each class separately
        process_images(train_folder, class_folders, centroids, os.getcwd(), prefix='train')
        process_images(test_folder, class_folders, centroids, os.getcwd(), prefix='test')
        
        print("BoVW representations saved for each class.")
    else:
        print("BoVW data already exists.")

if __name__ == "__main__":
    main()


Total training patches: 204224
K-means clustering completed.
BoVW representations saved for each class.


In [3]:
import pickle
import os

# Initialize empty lists to store train and test data for each class
train_data1, train_data2, train_data3 = [], [], []
test_data1, test_data2, test_data3 = [], [], []

# Loop through all pickle files in the current directory
for filename in os.listdir('.'):
    if filename.endswith('.pkl'):
        # Load the pickle file
        with open(filename, 'rb') as file:
            data = pickle.load(file)
        
        # Determine which class the file belongs to based on the filename
        if 'train_data1_' in filename:
            train_data1.append(data)
        elif 'train_data2_' in filename:
            train_data2.append(data)
        elif 'train_data3_' in filename:
            train_data3.append(data)
        elif 'test_data1_' in filename:
            test_data1.append(data)
        elif 'test_data2_' in filename:
            test_data2.append(data)
        elif 'test_data3_' in filename:
            test_data3.append(data)

print("Data concatenation and completed.")
print(len(train_data1))
print(len(train_data2))
print(len(train_data3))
print(len(test_data1))
print(len(test_data2))
print(len(test_data3))
train_labels1 = np.ones(len(train_data1))  # All samples in train_data1 are class 1
train_labels2 = np.ones(len(train_data2)) * 2  # All samples in train_data2 are class 2
train_labels3 = np.ones(len(train_data3)) * 3  # All samples in train_data3 are class 3

test_labels1 = np.ones(len(test_data1))  # All samples in test_data1 are class 1
test_labels2 = np.ones(len(test_data2)) * 2  # All samples in test_data2 are class 2
test_labels3 = np.ones(len(test_data3)) * 3  # All samples in test_data3 are class 3

# Concatenating the data
train_data = np.concatenate([train_data1, train_data2, train_data3], axis=0)
test_data = np.concatenate([test_data1, test_data2, test_data3], axis=0)

# Concatenating the labels
train_labels = np.concatenate([train_labels1, train_labels2, train_labels3], axis=0)
test_labels = np.concatenate([test_labels1, test_labels2, test_labels3], axis=0)
print(len(train_data))
print(len(train_labels))
print(len(test_data))
print(len(test_labels))

X_train = train_data
y_train = train_labels
X_test = test_data
y_test = test_labels

# Output to verify
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)



Data concatenation and completed.
50
50
50
50
50
50
150
150
150
150
X_train shape: (150, 32)
y_train shape: (150,)
X_test shape: (150, 32)
y_test shape: (150,)


In [4]:
def load_images_and_labels(base_path):
    """
    Load image data and corresponding labels from Dataset 2.

    Parameters:
    - base_path: Path to the dataset folder (train or test).

    Returns:
    - X: Flattened image data.
    - y: Corresponding labels.
    """
    X, y = [], []
    classes = os.listdir(base_path)  # Folder names represent classes
    for label, class_name in enumerate(classes):
        class_folder = os.path.join(base_path, class_name)
        for image_name in os.listdir(class_folder):
            image_path = os.path.join(class_folder, image_name)
            try:
                img = Image.open(image_path).convert('L').resize((32, 32))  # Resize to 32x32 and grayscale
                X.append(np.array(img).flatten())
                y.append(label)
            except Exception as e:
                print(f"Error loading image {image_path}: {e}")
    return np.array(X), np.array(y)



print(f"Train Data: {X_train.shape}, Train Labels: {y_train.shape}")
print(f"Test Data: {X_test.shape}, Test Labels: {y_test.shape}")


Train Data: (150, 32), Train Labels: (150,)
Test Data: (150, 32), Test Labels: (150,)


In [5]:
# Scale features using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Feature scaling completed.")


Feature scaling completed.


In [6]:
def tune_svm(X_train, y_train, kernel, param_grid):
    """
    Tune hyperparameters for SVM using GridSearchCV.

    Parameters:
    - X_train, y_train: Scaled training data and labels.
    - kernel: SVM kernel type.
    - param_grid: Dictionary of parameters to tune.

    Returns:
    - best_model: Best SVM model after tuning.
    - best_params: Best parameters.
    """
    svc = SVC(kernel=kernel)
    grid_search = GridSearchCV(svc, param_grid, scoring='accuracy', cv=5)
    grid_search.fit(X_train, y_train)
    return grid_search.best_estimator_, grid_search.best_params_

# Define parameter grids for each kernel
param_grids = {
    'linear': {'C': [0.1, 1, 10]},
    'poly': {'C': [0.1, 1, 10], 'degree': [2, 3]},
    'rbf': {'C': [0.1, 1, 10, 100], 'gamma': [0.01, 0.1, 1, 10]}
}

# Tune and store the best models for each kernel
best_models = {}
for kernel, param_grid in param_grids.items():
    print(f"Tuning SVM with {kernel} kernel...")
    best_model, best_params = tune_svm(X_train_scaled, y_train, kernel=kernel, param_grid=param_grid)
    best_models[kernel] = (best_model, best_params)
    print(f"Best Parameters for {kernel} kernel: {best_params}")


Tuning SVM with linear kernel...
Best Parameters for linear kernel: {'C': 0.1}
Tuning SVM with poly kernel...
Best Parameters for poly kernel: {'C': 1, 'degree': 2}
Tuning SVM with rbf kernel...
Best Parameters for rbf kernel: {'C': 1, 'gamma': 0.01}


In [7]:
def evaluate_svm(X_test, y_test, kernel, model):
    """
    Evaluate SVM for Dataset 2.

    Parameters:
    - X_test: Scaled testing data.
    - y_test: Corresponding labels.
    - kernel: SVM kernel type.
    - model: Trained SVM model.

    Returns:
    - Confusion matrix and classification metrics.
    """
    y_pred = model.predict(X_test)

    # Compute confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)

    # Compute classification metrics
    class_report = classification_report(y_test, y_pred, zero_division=0)

    print(f"SVM with {kernel} kernel:")
    print("Confusion Matrix:\n", conf_matrix)
    print("Classification Report:\n", class_report)
    
    return conf_matrix, class_report

# Evaluate each kernel and store results
evaluation_results = {}
for kernel, (model, params) in best_models.items():
    print(f"Evaluating SVM with {kernel} kernel...")
    conf_matrix, class_report = evaluate_svm(X_test_scaled, y_test, kernel=kernel, model=model)
    evaluation_results[kernel] = {
        'conf_matrix': conf_matrix,
        'class_report': class_report
    }


Evaluating SVM with linear kernel...
SVM with linear kernel:
Confusion Matrix:
 [[35  8  7]
 [ 4 41  5]
 [ 2  3 45]]
Classification Report:
               precision    recall  f1-score   support

         1.0       0.85      0.70      0.77        50
         2.0       0.79      0.82      0.80        50
         3.0       0.79      0.90      0.84        50

    accuracy                           0.81       150
   macro avg       0.81      0.81      0.80       150
weighted avg       0.81      0.81      0.80       150

Evaluating SVM with poly kernel...
SVM with poly kernel:
Confusion Matrix:
 [[36  1 13]
 [ 0 40 10]
 [ 0  0 50]]
Classification Report:
               precision    recall  f1-score   support

         1.0       1.00      0.72      0.84        50
         2.0       0.98      0.80      0.88        50
         3.0       0.68      1.00      0.81        50

    accuracy                           0.84       150
   macro avg       0.89      0.84      0.84       150
weighted avg   

In [8]:
# Save metrics for each kernel
for kernel, results in evaluation_results.items():
    print(f"Metrics for {kernel} kernel:")
    print(results['class_report'])
    
    # Extract precision, recall, and F1-score for each class
    y_pred = best_models[kernel][0].predict(X_test_scaled)
    precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average=None)
    metrics_df = pd.DataFrame({
        'Metric': ['Precision', 'Recall', 'F1-Score'],
        'Class 0': [precision[0], recall[0], f1[0]],
        'Class 1': [precision[1], recall[1], f1[1]],
        'Class 2': [precision[2], recall[2], f1[2]],
    })
    
    # Save metrics to CSV for reporting
    filename = f"{kernel}_kernel_metrics.csv"
    metrics_df.to_csv(filename, index=False)
    print(f"Metrics saved to {filename}")


Metrics for linear kernel:
              precision    recall  f1-score   support

         1.0       0.85      0.70      0.77        50
         2.0       0.79      0.82      0.80        50
         3.0       0.79      0.90      0.84        50

    accuracy                           0.81       150
   macro avg       0.81      0.81      0.80       150
weighted avg       0.81      0.81      0.80       150

Metrics saved to linear_kernel_metrics.csv
Metrics for poly kernel:
              precision    recall  f1-score   support

         1.0       1.00      0.72      0.84        50
         2.0       0.98      0.80      0.88        50
         3.0       0.68      1.00      0.81        50

    accuracy                           0.84       150
   macro avg       0.89      0.84      0.84       150
weighted avg       0.89      0.84      0.84       150

Metrics saved to poly_kernel_metrics.csv
Metrics for rbf kernel:
              precision    recall  f1-score   support

         1.0       0.89